# HW5 - Part 1

VAST challenge: [2011](http://hcil2.cs.umd.edu/newvarepository/VAST%20Challenge%202011/challenges/Grand%20Challenge%202011/)

Sections:
- [MC1](#Mini-Challenge-1)
- [MC2](#Mini-Challenge-2)
- [MC3](#Mini-Challenge-3)
- [Grand Challenge](#Grand-Challenge)

The setting for this VAST challenge is a major city. There have been increased reports of an illness spreading among residents and even sometimes resulting in death. Our goal is to provide an assessment of the situation.

In [10]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook

output_notebook()

Loading BokehJS ...

# Mini Challenge 1

This task is to characterize the spread of the epidemic. We need to find out where 

## Datasets:

The datasets for this challenge span across 4/30/2011 - 5/9/2011 and are:

| Type | Fields | Datapoints |
|------|--------|------------|
| Microblog messages with GPS | ID, Created_at, Location, text | 1023077 |
| Labeled map image | N/A | 1 |
| Population statistics | Zone_name, Population_Density, Daytime_Population | 13 |
| Observed weather | Date, Weather, Average_Wind_Speed, Wind_Direction | 25 |

## Machine Learning

### Topic Modeling

Part of the chall

### KNN Clustering

In [17]:
# Load VAST datasets
microblog_df = pd.read_csv('Datasets/MC_1/Microblogs.csv', encoding='ISO-8859-1')
population_df = pd.read_csv('Datasets/MC_1/Population.csv')
weather_df = pd.read_csv('Datasets/MC_1/Weather.csv')
map_img = 'Datasets/MC_1/Vastopolis_Map.png'
x_range = (-93.5673,-93.1923)
y_range = (42.1609,42.3017)

In [12]:
microblog_df.head()

,ID,Created_at,Location,text
0,3,5/18/2011 13:26,42.22717 93.33772,this convention filled with technology could b...
1,3,5/10/2011 9:22,42.18881 93.35642,4 orang top scorer skrg 3 di antara nya pake ...
2,3,5/14/2011 9:22,42.22479 93.35922,Nike 'gana' el Mundial al patrocinador oficial...
3,3,5/6/2011 9:22,42.2469 93.32527,Positiiff mau nabung beli kaos adidas aslii...
4,3,5/19/2011 9:22,42.2354 93.35642,I kick it like Adidas


In [23]:
population_df.head()

,Zone_Name,Population_Density,Daytime_Population
0,Cornertown,59524,59226
1,Downtown,89286,258928
2,Eastside,178571,125000
3,Lakeside,297619,181548
4,Northville,119048,95238


In [14]:
weather_df.head()

,Date,Weather,Average_Wind_Speed,Wind_Direction
0,4/30/2011,cloudy,9,E
1,5/1/2011,clear,5,E
2,5/2/2011,clear,4,SE
3,5/3/2011,showers,8,SE
4,5/4/2011,showers,10,N


In [20]:
p = figure(x_range=x_range, y_range=y_range)
p.image_url(url=[map_img],
            x=x_range[0],y=y_range[1],
            w=x_range[1]-x_range[0],h=y_range[1]-y_range[0])
show(p)

# Mini Challenge 2

This task is to conduct cyber security analysis for situational awareness of a corporate network.

## Datasets

The datasets for this challenge span across 4/30/2011 - 5/9/2011 and are:

| Type | Fields | Datapoints |
|------|--------|------------|
| File describing computer network architecture | N/A | N/A |
| Security policy rules | N/A | N/A |
| Firewall log | N/A | N/A |
| IDS log | N/A | N/A |
| Syslogs for all hosts on network | N/A | N/A |
| Nessus Network Vulnerability Scan Report | N/A | N/A |

## ML Algorithms

# Mini Challenge 3

The task for this challenge is to investigate terrorist activity in the region. 

## Datasets

| Type | Fields | Datapoints |
|------|--------|------------|
| File describing computer network architecture | N/A | N/A |
| Security policy rules | N/A | N/A |
| Firewall log | N/A | N/A |
| IDS log | N/A | N/A |
| Syslogs for all hosts on network | N/A | N/A |
| Nessus Network Vulnerability Scan Report | N/A | N/A |

## ML Algorithms

### Regression



# Grand Challenge

## Machine Learning

